### Mount drive to datasets:

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import libraries and set `DATA_PATH`:

In [7]:
from pathlib import Path
import json
import glob
import numpy as np
import librosa
import os
from librosa import display
import matplotlib.pyplot as plt
import tensorflow as tf
from time import strftime
assert tf.__version__ >= "2.0"
# autotune computation
AUTOTUNE = tf.data.experimental.AUTOTUNE
DATA_PATH = '/content/drive/MyDrive'

### All paths to possible codecs:
All codec datasets are created for a binary problem. <br>
See configs in DLNet_config.json for details.

In [8]:
# Path to datasets:
# Full dataset (with all codecs)
path_medley_train = os.path.join(DATA_PATH, 'tf_dataset_MedleyDB', 'MedleyDB_train_set')
path_medley_test = os.path.join(DATA_PATH, 'tf_dataset_MedleyDB', 'MedleyDB_test_set')

### Read `config` and create `wrapper` object:
Each dataset has a config file included with the dataset configurations.<br>
Dataset configurations have to match to concatenate the datasets later.

In [10]:
 # Read configs from dataset:
json_file_codec =  os.path.join(os.path.split(path_medley_train)[0], 'DLNet_config.json')
with open(json_file_codec, "r") as read_file:
            config = json.load(read_file)
# Print Config:
print(json.dumps(config, sort_keys=False, indent=4))

{
    "audio_length": 1,
    "binary": false,
    "calculate_mel": false,
    "center": true,
    "classes": [
        "mp3_32k",
        "mp3_160k",
        "mp3_192k",
        "mp3_320k",
        "aac_128",
        "ogg_vbr",
        "uncompr_wav"
    ],
    "filter_signal": false,
    "hop_length": 256,
    "input_shape": [
        513,
        173,
        1
    ],
    "mono": true,
    "n_fft": 1024,
    "n_frames": 173,
    "n_mels": 64,
    "pad_mode": "reflect",
    "power": 2.0,
    "random_seed": 10,
    "sr": 44100,
    "time_stamp": "14_02_2021_16_41",
    "win_length": 512,
    "window": "hamm"
}


### Load chosen datasets and concatenate them:

In [11]:
# Load dataset:
# Codec1: (Compressed)
train_dataset = tf.data.experimental.load(path_medley_train,
                        (tf.TensorSpec(config['input_shape'],
                                       dtype=tf.float32, name=None),
                         tf.TensorSpec(len(config['classes']),
                                       dtype=tf.uint8, name=None)),
                        compression='GZIP')
test_dataset = tf.data.experimental.load(path_medley_test,
                        (tf.TensorSpec(config['input_shape'],
                                       dtype=tf.float32, name=None),
                         tf.TensorSpec(len(config['classes']),
                                       dtype=tf.uint8, name=None)),
                        compression='GZIP')


### Print dataset info:

In [13]:
print(train_dataset)
print(len(train_dataset))
print(test_dataset)
print(len(test_dataset))

<_LoadDataset shapes: ((513, 173, 1), (7,)), types: (tf.float32, tf.uint8)>
23200
<_LoadDataset shapes: ((513, 173, 1), (7,)), types: (tf.float32, tf.uint8)>
6000


### Prepare datasets:


*   Shuffle `train_dataset`
*   Take 10 % of `train_dataset` for evaluation: `eval_dataset`
*   Shuffle and batch rest of `train_dataset`
*   Batch `test_dataset`
*   Batch size: 64



In [14]:
train_size = len(train_dataset)
test_size = len(test_dataset)
eval_size = int(.1*train_size)
batch_size = 64
scaling_factor = .5

# Shuffel train data:
train_dataset = train_dataset.shuffle(buffer_size=int(train_size/3))

# Split train into train and eval set:
eval_dataset = train_dataset.take(int(scaling_factor*eval_size))
eval_dataset = eval_dataset.batch(batch_size).prefetch(AUTOTUNE)

# Train dataset
train_dataset = train_dataset.skip(eval_size)
train_dataset = train_dataset.take(int(scaling_factor*train_size))
train_dataset = train_dataset.shuffle(train_size - eval_size)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(AUTOTUNE)

# Prepare test dataset
test_dataset = test_dataset.take(int(scaling_factor*test_size)).batch(batch_size).prefetch(AUTOTUNE)


### Create and compile CNN Model:

In [15]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=config['input_shape']))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.GlobalMaxPool2D())
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.GaussianDropout(0.25))
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.GaussianDropout(0.25))
model.add(tf.keras.layers.Dense(len(config['classes']), activation="softmax"))

# Define metrics
metrics = [tf.keras.metrics.TrueNegatives(),
           tf.keras.metrics.TruePositives(),
           tf.keras.metrics.FalseNegatives(),
           tf.keras.metrics.FalsePositives(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           tf.keras.metrics.CategoricalAccuracy()
           ]

# compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=metrics)

### Train and evaluate model:

In [ ]:
# fit model
n_epochs = 17
history = model.fit(train_dataset, epochs=n_epochs,
                    validation_data=eval_dataset, verbose=1)

model.evaluate(test_dataset, batch_size=64)

Epoch 1/17


In [1]:
# setup plot
fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(16,4))

# plot loss
ax[0].plot(range(n_epochs), history.history['loss'])
ax[0].plot(range(n_epochs), history.history['val_loss'])
ax[0].set_ylabel('loss'), ax[0].set_title('train_loss vs val_loss')

# plot accuracy
ax[1].plot(range(n_epochs), history.history['accuracy'])
ax[1].plot(range(n_epochs), history.history['val_accuracy'])
ax[1].set_ylabel('accuracy'), ax[1].set_title('train_acc vs val_acc')

# plot adjustement
for a in ax:
    a.grid(True)
    a.legend(['train','val'], loc=4)
    a.set_xlabel('num of Epochs')
plt.show()


NameError: ignored

### Model prediction as numpy arrays:

In [ ]:
y_test_prob = model.predict(test_dataset)
y_test_pred = np.argmax(y_test_prob, axis=1)
y_test_true = np.argmax(np.array([y for x, y in test_dataset.unbatch().as_numpy_iterator()]), axis=1)

### Print metrics with sklearn:

In [ ]:
from sklearn.metrics import classification_report

print('Classification report:')
print(classification_report(y_true=y_test_true, y_pred=y_test_pred,
                            target_names=config['classes']))

print('------------------------------------------------:')
print('Evaluate model on test dataset:')
hist_eval = model.evaluate(test_dataset)

print('------------------------------------------------:')
print('Evaluate model on eval dataset:')
hist_eval = model.evaluate(eval_dataset)

### Confusion Matrix:

In [ ]:
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label'), plt.xlabel('Predicted label')

# call confusion matrix
cm = tf.math.confusion_matrix(labels=y_test_true, predictions=y_test_pred)
cm = cm.numpy()
plot_confusion_matrix(cm, classes=config['classes'], normalize=False,
                      title='Confusion matrix', cmap=plt.cm.Blues)